# Project: Llama 2 Inference

This notebook experiments will the generative model trained in the notebook `train.ipynb`.

Set up the environment.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/projects/llama_2/

Mounted at /content/drive
/content/drive/MyDrive/projects/llama_2


In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1.0 --progress-bar off
!pip install -qqq transformers==4.32.1 --progress-bar off
!pip install -qqq datasets==2.14.4 --progress-bar off
!pip install -qqq peft==0.5.0 --progress-bar off
!pip install -qqq bitsandbytes==0.41.1 --progress-bar off
!pip install -qqq trl==0.7.1 --progress-bar off
!pip install -qqq xmltodict --progress-bar off

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import PeftModel, AutoPeftModelForCausalLM
import torch

Load the tokenizer.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained( "meta-llama/Llama-2-7b-hf" )

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Load the model.

In [ ]:
model_id = 'RickMartel/Llama-2-7b-hf'
model = AutoPeftModelForCausalLM.from_pretrained(model_id)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

In [ ]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
model.to( DEVICE )

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear(
     

In [ ]:
merged_model = model.merge_and_unload()

Generate text.

In [ ]:
%%time
import torch

prompt = tokenizer.bos_token + "Jesus said to them"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to( DEVICE )

out = merged_model.generate(
    generated,
    max_new_tokens=100,)
out = tokenizer.decode(out[0], skip_special_tokens=True)
from IPython.display import display, Markdown
display(Markdown(out))


Jesus said to them, "If God were your Father, you would love me, for I came from God and I am here. I came not of my own accord, but he sent me. Why do you not understand what I say? It is because you cannot bear to hear my word. You are of your father the devil, and your will is to do your father's desires. He was a murderer from the beginning, and does not stand in the truth, because there is no truth in

CPU times: user 3.48 s, sys: 0 ns, total: 3.48 s
Wall time: 3.47 s
